# PRACTICA GUIADA 2: Selección e Indexado de Datos en Pandas

* Recordemos algunas formas típicas de acceder a los arrays:
        
  1. indexing: `arr[2,1]`
  2. slicing: `arr[:,1:10]`
  3. boolean indexing: `arr[arr>0]`
  4. fancy indexing: `arr[[1,7,9],:]`
  
  
  
* Las `Series` y `DataFrames` de Pandas siguen convenciones similares.         

## Selección de Datos en Series

* Si recordamos que una `Series` es un análogo a un array de una dimensión y a un diccionario esto nos va a permitir retener mejor la forma de selccionar datos.  

### `Series` como un diccionario

* Indexar por nombres (=key en diccionarios)

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

* Podemos usar expresiones similares a los dicts para examinar keys y valores.

In [2]:
'b' in data

True

In [3]:
# keys() es un método que nos trae el index:

data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [4]:
# Podemos llamar al index directamente invocando el atributo:

data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

* Como en un dict podemos extender una `Series` definiendo una nueva key y asignarle un nuevo valor

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

### `Series` como un array de una dimensión

* Una `Series` provee una forma de seleccionar datos análoga a los arrays: por eso podemos usar _slices_, _masking_ y _fancy indexing_.

In [7]:
data['a':'c'] # slicing explícito

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
data[0:2] # slicing implícito por posición (enteros)

a    0.25
b    0.50
dtype: float64

In [9]:
data[(data > 0.3) & (data < 0.8)] # boolean masking

b    0.50
c    0.75
dtype: float64

In [10]:
data[['a', 'e']] # fancy indexing

a    0.25
e    1.25
dtype: float64

In [11]:
data[['a', 'e', 'e', 'b']] # otro ejemplo de fancy indexing

a    0.25
e    1.25
e    1.25
b    0.50
dtype: float64

In [12]:
data2 = data.reindex(['d', 'b', 'a', 'c','d', 'b', 'a', 'c', 'e', 'e']) # reindexing
data2

d    1.00
b    0.50
a    0.25
c    0.75
d    1.00
b    0.50
a    0.25
c    0.75
e    1.25
e    1.25
dtype: float64

In [13]:
data3 = data.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g'], method='ffill') # otro ejemplo
data3                                                                    #de reindexing

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
f    1.25
g    1.25
dtype: float64

### Indexers: loc e iloc

* Posible confusión: 

    - cuando se hace slicing explícito (`data['a':'c']`) el índice final es incluido en el slice. 
    - en cambio, cuando se hace slicing implícto (`data[0:2]`) el índice final NO es incluido

* Para mitigar este tipo de confusiones, Pandas provee algunos atributos "indexadores".

** Método `loc`** 

In [14]:
data.loc['a']

0.25

In [15]:
data.loc['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

** Método `iloc`** 

In [16]:
data.iloc[1]

0.5

In [17]:
data.iloc[0:3]

a    0.25
b    0.50
c    0.75
dtype: float64

## Selección de datos en `DataFrame`

### DataFrame como un diccionario

In [18]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127
Texas,695662,26448193


* Puede accederse a los primeros n elementos del DataFrame con el método df.head(n). Del mismo modo, puede aplicarse el método df.tail(n) para acceder a los últimos elementos del DataFrame:

In [19]:
data.head(2)

,area,pop
California,423967,38332521
Florida,170312,19552860


In [20]:
data.tail(3)

,area,pop
Illinois,149995,12882135
New York,141297,19651127
Texas,695662,26448193


* Con el método df.sample(n) traemos una muestra aleatória de n elementos:

In [21]:
data.sample(2)

,area,pop
Texas,695662,26448193
Florida,170312,19552860


* Puede accederse a las ``Series`` individuales que forman las columnas del ``DataFrame`` de forma análoga a un diccionario, vía el nombre de la columna.

In [22]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

* De forma equivalente, podemos acceder a la columna como atributo:

In [23]:
data.area

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

* Ambas formas son equivalentes.
* ¿Qué pasaría si hay algún espacio en el nombre de la columna?

In [24]:
data.area is data['area']

True

* Tener en cuenta que esta forma no siempre funciona. 

    - Por ejemplo, si los nombres de las columnas no son strings
    -  o si tienen nombres que entran en conflicto on algún método de `DataFrame`
  

* Ejemplo: el `DataFrame` tiene un método `pop()`, de esta forma, `data.pop` apuntará al método y no a la columna de `data`  

In [25]:
data.pop is data['pop']

False

* En particular, es importante evitar la asignación de columnas vía atributos (usar `data['pop'] = z` en lugar de `data.pop = z`)
* El estilo diccionario puede ser usado para modificar un objeto:

In [26]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


### DataFrame como un array bi-dimensional

* Examinmenos el atributo `values`

In [27]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01]])

* Teniendo en cuenta esto, podemos realizar la analogía y utilizar muchas operaciones similares a la de los arrays en un `DataFrame`.

# * Al igual que en el caso de una `Series` indexar un `DataFrame` de forma análoga a un array puede ser un tanto confuso.

* Particularmente, pasar un índice simple en un `DataFrame` devuelve una fila. 

In [28]:
data[0:1]
# data[0] da error

,area,pop,density
California,423967,38332521,90.413926


* Y pasar un índice simple devuelve una columna:

* Por eso Pandas usa los indexadores `loc` e `iloc`.

* Usando `iloc` podemos indexar los arrays subyacentes a un `DataFrame` como si fuera un array común, pero el índice y la etiqueta de columna son mantenidos en el resultado:

In [29]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


* De forma similar, usando `loc` podemos indexar el array subyancente pero usando el index de forma explícita y los nombre de columnas.

In [30]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


* Cualquier forma de acceso de un array puede usarse con estos indexadores.
* Por ejemplo, podemos usar `loc` y combinarlo con masking y fancy indexing:

In [31]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Florida,19552860,114.806121
New York,19651127,139.076746


* Cualquiera de estas formas de indexar puede ser usada para asignar o modificar valores:

In [32]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


### Algunas convenciones adicionales para indexar

* En general, "indexing" refiere a columnas, mientras que "slicing" refiere a filas:

In [33]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


* "Fancy indexing" por defecto se realiza de forma explícita y sobre las columnas.

In [34]:
data[['area','density']]

,area,density
California,423967,90.000000
Florida,170312,114.806121
Illinois,149995,85.883763
New York,141297,139.076746
Texas,695662,38.018740


* Esos slices también pueden referir a filas por posición, en lugar de índices:

In [35]:
data[1:3]

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


* De forma similar, las operaciones de masking también son interpretadas por defecto en el sentido de las filas:

In [36]:
data[data.density > 100]

,area,pop,density
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746


In [37]:
data.reindex(['California', 'Florida', 'New York', 'Texas', 'Florida', 'New York', 'Texas'])

,area,pop,density
California,423967,38332521,90.000000
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


In [38]:
data.reindex(columns = ['pop', 'density', 'area', 'pop'])

,pop,density,area,pop
California,38332521,90.000000,423967,38332521
Florida,19552860,114.806121,170312,19552860
Illinois,12882135,85.883763,149995,12882135
New York,19651127,139.076746,141297,19651127
Texas,26448193,38.018740,695662,26448193


In [39]:
data.reindex(index= ['California', 'Florida', 'New York',\
                      'Texas', 'Florida', 'New York', 'Texas'],\
                      columns = ['pop', 'density', 'area', 'pop'])

,pop,density,area,pop
California,38332521,90.000000,423967,38332521
Florida,19552860,114.806121,170312,19552860
New York,19651127,139.076746,141297,19651127
Texas,26448193,38.018740,695662,26448193
Florida,19552860,114.806121,170312,19552860
New York,19651127,139.076746,141297,19651127
Texas,26448193,38.018740,695662,26448193
